In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# !pip install fuzzywuzzy
# !pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib.pyplot as plt
import seaborn as sns


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

## Premier test de cohérence de la donnée entre les datasets: voyons si les colonnes "freedom to make life choices" (World Happiness Report) et "personal_freedom" (Economic Freedom) disent la même chose

In [ ]:
df_ef = pd.read_csv('/kaggle/input/the-human-freedom-index/hfi_cc_2019.csv')
df_h = pd.read_csv('/kaggle/input/world-happiness/2018.csv')


### Premier problème, les deux listes de pays ne sont pas exactement les mêmes (par exemple South Africa vs South African Rep.) La librairie FuzzyWuzzy permet de trouver les équivalences.

In [ ]:
## On ne garde que les données les plus récentes (2017) 

combined_df = df_ef[df_ef["year"]==2017]
combined_df = combined_df[["countries","pf_score","ef_legal_courts"]]
combined_df.columns = ['countries_ef', 'personal_freedom_ef','corruption_ef']

tempdf_h = pd.DataFrame()
tempdf_h[["countries_hr","corruption_hr","personal_freedom_hr"]] = df_h[["Country or region","Perceptions of corruption","Freedom to make life choices"]]

# On définit la masterlist en utilisant les pays présents dans l'indice Human Freedom, qui est plus complet
masterlist_countries = combined_df["countries_ef"].tolist()

## En utilisant FuzzyWuzzy, on regarde, pour chaque pays dans l'indice , si son équivalent est présent dans la masterlist et on harmonise.    
for i in tempdf_h.index: 
    fuzz_score = process.extract(str(tempdf_h.loc[i,"countries_hr"]), masterlist_countries, scorer = fuzz.token_sort_ratio,limit=1)
    if fuzz_score[0][1]>70:
        tempdf_h.loc[i,"countries_ef"] = fuzz_score[0][0]
    else:
        tempdf_h.loc[i,"countries_ef"] = "country doesn't exist in EF dataset"  
tempdf_h = tempdf_h.set_index(["countries_ef"])

combined_df = combined_df.merge(tempdf_h, right_on="countries_ef",left_on="countries_ef")    
    
display(combined_df)

### Voyons  si les données sur la liberté présentes dans les deux datasets concordent 

In [ ]:
combined_df[['personal_freedom_ef','personal_freedom_hr']] = combined_df[['personal_freedom_ef','personal_freedom_hr']].astype(float)

combined_df.plot(kind='scatter', x='personal_freedom_ef', y='personal_freedom_hr')

### Même chose pour la corruption (dataset "World Happiness") et l'indicateur "impartialité des tribunaux" (dataset Human Freedom)

In [ ]:
combined_df[['corruption_ef','corruption_hr']] = combined_df[['corruption_ef','corruption_hr']].astype(float)
combined_df.plot(kind='scatter', x='corruption_ef', y='corruption_hr')

## Bilan: les données sont loin d'être superposées, bien qu'il y ait une corrélation positive. Ce sont des données très subjectives et donc difficiles à mesurer : il est normal que deux méthodologies différentes donnent des résultats différents. Il est important de garder cette limite en tête pour toute analyse ultérieure.

## Analysons maintenant les déterminants du bonheur : le dataset World Happiness Report

In [ ]:
path = '../input/world-happiness'  # path to data in Kaggle notebook
Happinness = pd.read_csv(os.path.join(path, '2017.csv'))


### Présentation du dataset

In [ ]:
print(Happinness.info())
display(Happinness.head())

#### 155 pays (valeurs catégoriques nominales) ayant chacune un happiness score composé de 7 sous-indicateurs (6 sous-indicateurs véritables - Famille, Générosité...- et un sous-indicateur qui est une construction statistique - Dystopia, qui représente le pire pays du monde).
+ Un classement des pays selon leur Happiness score
+ Des Whiskers qui représentent l'intervalle de confiance statistique associé à la mesure du happinness score.

#### Les données sont complètes, le nombre de dimensions est gérable: nous allons tous les utiliser. 

In [ ]:
Happinness["Country"].unique()


In [ ]:
# get all country/region names
countries = set(Happinness['Country'])
print('The dataset contains {} unique countries'.format(len(countries))) 

In [ ]:
# generate empty column 'Continent'
Happinness.insert(loc=1, column='Continent', value="")
Happinness.columns


In [ ]:
asia = ["Israel", "United Arab Emirates", "Singapore", "Thailand", "Taiwan Province of China",
                                   "Qatar", "Saudi Arabia", "Kuwait", "Bahrain", "Malaysia", "Uzbekistan", "Japan",
                                   "South Korea", "Turkmenistan", "Kazakhstan", "Turkey", "Hong Kong S.A.R., China", "Philippines",
                                   "Jordan", "China", "Pakistan", "Indonesia", "Azerbaijan", "Lebanon", "Vietnam",
                                   "Tajikistan", "Bhutan", "Kyrgyzstan", "Nepal", "Mongolia", "Palestinian Territories",
                                   "Iran", "Bangladesh", "Myanmar", "Iraq", "Sri Lanka", "Armenia", "India", "Georgia",
                                   "Cambodia", "Afghanistan", "Yemen", "Syria"]
europe = ["Norway", "Denmark", "Iceland", "Switzerland", "Finland",
                                   "Netherlands", "Sweden", "Austria", "Ireland", "Germany",
                                   "Belgium", "Luxembourg", "United Kingdom", "Czech Republic",
                                   "Malta", "France", "Spain", "Slovakia", "Poland", "Italy",
                                   "Russia", "Lithuania", "Latvia", "Moldova", "Romania",
                                   "Slovenia", "North Cyprus", "Cyprus", "Estonia", "Belarus",
                                   "Serbia", "Hungary", "Croatia", "Kosovo", "Montenegro",
                                   "Greece", "Portugal", "Bosnia and Herzegovina", "Macedonia",
                                   "Bulgaria", "Albania", "Ukraine"]
northamerica = ["Canada", "Costa Rica", "United States", "Mexico",  
                                   "Panama","Trinidad and Tobago", "El Salvador", "Belize", "Guatemala",
                                   "Jamaica", "Nicaragua", "Dominican Republic", "Honduras",
                                   "Haiti"]
southamerica = ["Chile", "Brazil", "Argentina", "Uruguay",
                                   "Colombia", "Ecuador", "Bolivia", "Peru",
                                   "Paraguay", "Venezuela"]
australia = ["New Zealand", "Australia"]
africa = ['South Africa', 'Benin', 'Congo (Brazzaville)', 'Rwanda', 'Guinea', 'Nigeria', 
          "Ivory Coast", 'Morocco', 'Somalia', 'Algeria', 'Tanzania', 'Ghana',
          'Mauritia', 'Egypt',  'Liberia', 'Congo (Kinshasa)', 'Zambia', 'Ethiopia', 'Namibia', 'Sudan', 'Togo', 
          'Burkina Faso', 'Tunisia', 'Central African Republic', 'Mauritania', 'Cameroon',
          'Senegal', 'Kenya', 'Gabon', 'Mauritius', 'Sierra Leone', 'Namibia', 'Mozambique','Ethiopia' ,'Uganda' 'Malawi', 'Chad', 
          'Zimbabwe', 'Niger', 'Lesotho', 'Angola', 'Botswana', 'Madagascar', 'South Sudan', 'Burundi']


In [ ]:
# generate dictionary
continents = {'Asia': asia,
              'Europe': europe,
              'South America': southamerica,
              'North America': northamerica,
              'Africa': africa,
              'Australia': australia}

In [ ]:
for i in range(Happinness.shape[0]):
    try:
        country = Happinness['Country'][i]
        continent = [k for k, v in continents.items() if country in v][0]
        Happinness['Continent'][i] = continent
    except IndexError:
        pass

In [ ]:
Happinness.drop(['Whisker.high', 'Whisker.low', 'Happiness.Rank', 'Dystopia.Residual'], axis=1, inplace=True)
Happinness.head()

### Le graphique ci-dessous nous montre que l'économie à un impacte positive sur le bonheur. Nous constatons que l'Europe se trouve au dessus des autres continents. 

In [ ]:
sns.scatterplot(x = 'Economy..GDP.per.Capita.', y = 'Happiness.Score', data= Happinness, hue = 'Continent')

### Le graphique ci-dessous, permet de voir les variables les plus colerrées et les moins correlées.  Nous observons que l'économie (PIB par habitant), la sante et la famille jouent le rôle le plus important dans la contribution au bonheur. La confiance et la générosité ont le plus faible impact sur le score de bonheur.

In [ ]:
    sns.heatmap(Happinness.corr())

## OK, nous avons maintenant percé le secret du bonheur ! Mais peut-on être heureux sans être libre ? Penchons nous sur le Human Freedom Index. 


In [ ]:
df= pd.read_csv('../input/the-human-freedom-index/hfi_cc_2019.csv',na_values= ['-'])
df.info()
df.dtypes

#### Un dataset assez difficile à gérer :
- ~150 pays
- 9 années
- 120 points de données par an par pays

Chaque pays reçoit un score de "human freedom", composé de deux sous-indictauers (personal freedom et economic freedom), eux-mêmes composés d'une soixantaine de sous-sous-indicateurs (allant du poids du gouvernement dans l'économie aux discriminations contre les homosexuels à la liberté de culte), chacun ayant un score sur 10.

Pays, région : variables catégoriques nominales
Codes iso : variables catégoriques numériques
Indicateurs : variables numériques


# Attention : le biais idéologique du dataset est fort : les think tanks qui le produisent sont très alignés avec la droite américaine.


## Impossible de faire remonter les informations pertinentes avec 120 colonnes. Plutôt que de faire le tri nous-mêmes , réduisons la taille du dataset en identifiant les sous-indicateurs les plus corrélés avec le score total


In [ ]:
## Nous enlevons les données (l'année, sous-indicateurs intermédiaires, nom du pays...) qui ne rentrent pas dans le calcul du score 
df1= df.drop(columns=['year','ISO_code', 'countries', 'region','hf_rank','pf_rank', 'pf_score','ef_score','ef_rank', 'hf_quartile'])

# Affichons les 15 colonnes ayant la plus haute corrélation avec le score 
corr_matrix=df1.corr()
top_15_corr_positive = corr_matrix['hf_score'].sort_values(ascending=False)[:15]
top_15_corr_positive

### Il sera également intéressant de savoir ce qui enlève à la liberté des hommes. Voyons également quelles sont les 15 variables présentant les corrélations négatives.

In [ ]:
top_15_corr_negative= corr_matrix['hf_score'].sort_values(ascending=True)[:15]
top_15_corr_negative

## Trente variables, c'est trop ! Ne gardons que les corrélations négatives et les plus importantes corrélations positives.

#### Dans un monde idéal avec un peu plus de temps, il faudrait faire un tri qualitatif pour s'assurer que ces variables ne mesurent pas la même chose (par exemple, garder à la fois la variable "contrôle des médias" et la variable "influence du pouvoir sur les médias" n'est pas forcément pertinent)

In [ ]:
df_hfi= df[['year','countries','hf_score','pf_rol_procedural', 'pf_rol','ef_legal','ef_trade','pf_expression_control','pf_expression_influence',    
'pf_expression',             
'pf_ss',                      
'pf_rol_criminal',            
'pf_rol_civil',               
'ef_trade_regulatory',        
'pf_movement',               
'ef_legal_military',         
'ef_money',
'ef_government_transfers',        
'ef_government_consumption',      
'ef_government_tax_payroll',      
'ef_government_tax',              
'ef_government_tax_income']]

df_hfi

In [ ]:
plt.figure(figsize=(14,10))

plt.title("CORRELATION HEATMAP",fontsize=20)
sns.heatmap(data=df_hfi.drop(['year'], axis=1).corr(),cmap="PRGn_r",annot=True, fmt='.2f', linewidths=1)
plt.show()

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, Slider
from bokeh.palettes import cividis,Pastel1
from bokeh.transform import factor_cmap
from bokeh.layouts import column
from bokeh.themes import Theme
from math import pi
from bokeh.models.widgets import Panel, Tabs
import pandas as pd
from bokeh.models import HoverTool

import yaml

output_notebook()

In [ ]:
df= pd.read_csv('../input/the-human-freedom-index/hfi_cc_2019.csv',na_values= ['-'])

df_rank=df[['hf_rank','year','countries', 'pf_score','ef_score', 'hf_score']]
df_rank
  
df_vbar = df_rank.sort_values(by=["year", "hf_rank"], ascending=[True,True ]) 



In [ ]:
def bkapp(doc):
    source = ColumnDataSource(data={
        'countries': df_vbar.countries[df_vbar['year'] == 2008].head(15),
        'ef_score': df_vbar.ef_score[df_vbar['year'] == 2008].head(15),
        'pf_score': df_vbar.pf_score[df_vbar['year'] == 2008].head(15),
        'hf_score': df_vbar.hf_score[df_vbar['year'] == 2008].head(15),
        'hf_rank': df_vbar.hf_rank[df_vbar['year'] == 2008].head(15),
    })

    #recuper une liste des pays (utilisé pour la palette de couleur)
    countries = source.data['countries'].tolist()
    color_map = factor_cmap(field_name='countries',
                        palette=cividis(16), factors=countries)


    p3 = figure(x_range=countries, width=1800, plot_height=1000 )
    
    ## onglet 1: Human Freedom 
    
    p3.vbar(x='countries', top='hf_score', source=source, width=0.70, color=color_map, 
           line_color="yellow")

    # changement sur le titres
    p3.title.text_color = "red"
    p3.title.text_font = "times"
    p3.title.text_font_style = "italic"
    p3.xaxis.major_label_orientation = pi/4
    # changement sur les axis 
    p3.xaxis.axis_label = 'Country'
    p3.xaxis.axis_label_text_color = "red"
    p3.yaxis.major_label_text_color = "red"

     # Hoover
    hover = HoverTool(tooltips=[('Country', '@countries'), ('ef_score', '@ef_score'), ('pf_score', '@pf_score'), ('hf rank', '@hf_rank')])
    p3.add_tools(hover)

    ## onglet 2: Economic Freedom 
    p = figure(x_range=countries, width=1800, plot_height=1000 )
    p.vbar(x='countries', top='ef_score', source=source, width=0.70, color=color_map, 
           line_color="yellow")

    # changement sur le titres
    p.title.text_color = "red"
    p.title.text_font = "times"
    p.title.text_font_style = "italic"
    p.xaxis.major_label_orientation = pi/4
    # changement sur les axis 
    p.xaxis.axis_label = 'Country'
    p.xaxis.axis_label_text_color = "red"
    p.yaxis.major_label_text_color = "red"

     # Hoover
    hover = HoverTool(tooltips=[('Country', '@countries'), ('ef_score', '@ef_score'), ('pf_score', '@pf_score'), ('hf rank', '@hf_rank')])
    p.add_tools(hover)
  
    # slider pour faire defiler les années
    slider = Slider(start=2008, end=2017, step=1, value=2008, title='Year')

    # fonction pour updater les données
    def update_plot(attr, old, new):
        # yr= année and update les donneée => source.data = new_data`
        yr = slider.value

        new_data = {
        'countries': df_vbar.countries[df_vbar['year'] == yr].head(15),
        'ef_score': df_vbar.ef_score[df_vbar['year'] == yr].head(15),
        'pf_score': df_vbar.pf_score[df_vbar['year'] == yr].head(15),
        'hf_score': df_vbar.hf_score[df_vbar['year'] == yr].head(15),
        'hf_rank': df_vbar.hf_rank[df_vbar['year'] == yr].head(15),
        }

        source.data = new_data
        countries1= list(df_vbar.countries[df_vbar['year'] == yr].head(15))
        p.x_range.factors=countries1
        p2.x_range.factors=countries1
        p3.x_range.factors=countries1

        # Add title to figure: plot.title.text
        p.title.text = 'Economic Freedom rank for %d' % yr
        p2.title.text = 'Private Freedom rank for %d' % yr
        p3.title.text = 'Human Freedom rank for %d' % yr



    # appel callback avec slider.value
    slider.on_change('value', update_plot)


    # onglet 3: Private freedom
    p = figure(x_range=countries, width=1800, plot_height=1000 )
    p2 = figure(x_range=countries, width=1800, plot_height=1000 )
    p2.vbar(x='countries', top='pf_score', source=source, width=0.70, color=color_map, 
           line_color="yellow")

    # changement sur le titres
    p2.title.text_color = "red"
    p2.title.text_font = "times"
    p2.title.text_font_style = "italic"
    p2.xaxis.major_label_orientation = pi/4
    # changement sur les axis 
    p2.xaxis.axis_label = 'Country'
    p2.xaxis.axis_label_text_color = "red"
    p2.yaxis.major_label_text_color = "red"

     # hover
    hover = HoverTool(tooltips=[('Country', '@countries'), ('ef_score', '@ef_score'), ('pf_score', '@pf_score'), ('hf rank', '@hf_rank')])
    p2.add_tools(hover)
  
   
    tab1 = Panel(child=p,title="Economic Freedom rank")
    tab2 = Panel(child=p2, title="Private Freedom rank")
    tab3 = Panel(child=p3, title="human Freedom rank")
    tabs = Tabs(tabs=[ tab3, tab2, tab1 ])

    doc.add_root(column(slider, tabs))

    doc.theme = Theme(json=yaml.load("""
            attrs:
                Figure:
                    background_fill_color: "#DDDDDD"
                    outline_line_color: white
                    toolbar_location: above
                    height: 500
                    width: 800
                Grid:
                    grid_line_dash: [6, 4]
                    grid_line_color: white
        """, Loader=yaml.FullLoader))
    
show(bkapp)

## Evolution dans le temps des libertés humaines (telles que définies par l'indice Human Freedom...) : le monde devient-il plus libre ?




In [ ]:
df_ef_2 = pd.read_csv('/kaggle/input/the-human-freedom-index/hfi_cc_2019.csv',na_values="-")
df_ef_2 = df_ef_2[["year","region","countries","hf_score"]]
df_ef_2["hf_score_2017"] = df_ef_2["hf_score"].astype(float)
df_ef_2017 = df_ef_2[df_ef_2["year"]==2017]
df_ef_2008 = df_ef_2[df_ef_2["year"]==2008]

for i in df_ef_2017["countries"].unique():
    df_ef_2017.loc[df_ef_2017["countries"] == i, "hf_score_2008"] = df_ef_2008.loc[df_ef_2008["countries"] == i, "hf_score"].values
df_ef_2017 = df_ef_2017[(df_ef_2017["hf_score_2008"].notnull()) & (df_ef_2017["hf_score_2017"].notnull())]
    
df_ef_2017 = df_ef_2017.drop(['hf_score','year'],1)

df_ef_2017

In [ ]:
from pandas.plotting import parallel_coordinates
import seaborn as sns
sns.set(color_codes=True)
sns.set_palette('colorblind')


# On recatégorise dans deux "super catégories" pour garder le graphe lisible
df_ef_2017.loc[df_ef_2017["hf_score_2008"]>=df_ef_2017["hf_score_2008"].median(), "Pays libre en 2008"] = "Pays déjà libre en 2008"
df_ef_2017.loc[df_ef_2017["hf_score_2008"]<df_ef_2017["hf_score_2008"].median(), "Pays libre en 2008"] = "Pays déjà non-libre en 2008"

fig = plt.figure(figsize=(20, 15))
title = fig.suptitle("Evolution de l'indice Human Freedom pour chaque pays entre 2008 et 2017", fontsize=18)
fig.subplots_adjust(top=0.93, wspace=0)

pc = parallel_coordinates(df_ef_2017[["hf_score_2008","hf_score_2017",'Pays libre en 2008']],
                          'Pays libre en 2008',
                          color = ('r','g'))
